## Data Analytics

In [34]:
import boto3
import pandas as pd
import json
from io import StringIO

### S3 client pointing to LocalStack

In [35]:

s3 = boto3.client(
    "s3",
    endpoint_url="http://localhost:4566",
    aws_access_key_id="test",
    aws_secret_access_key="test",
    region_name="us-east-1"
)

 ### Loading CSV Data from S3 (LocalStack)

In [36]:
csv_obj = s3.get_object(Bucket="bls-data", Key="pub/time.series/pr/pr.data.0.Current")
csv_data = csv_obj["Body"].read().decode("utf-8")
df_csv_data = pd.read_csv(StringIO(csv_data), delimiter="\t")
df_csv_data.columns = df_csv_data.columns.str.strip()


 ### Loading JSON Data from S3 (LocalStack)

In [38]:
json_obj = s3.get_object(Bucket="bls-data", Key="api/datausa_population.json")
json_data = json.loads(json_obj["Body"].read())
df_population = pd.json_normalize(json_data['data'])

### Clean and filter population data

In [39]:
df_population["Year"] = df_population["Year"].astype(int)
df_population["Population"] = df_population["Population"].astype(int)

population_filtered = df_population[(df_population["Year"] >= 2013) & (df_population["Year"] <= 2018) & (df_population["Nation"] == "United States")]

### US Population Mean & Standard Deviation (2013–2018)

In [40]:
mean_pop = population_filtered["Population"].mean()
stddev_pop = population_filtered["Population"].std()

print("Mean Population (2013–2018):", mean_pop)
print("Std Dev Population (2013–2018):", stddev_pop)

Mean Population (2013–2018): 317437383.0
Std Dev Population (2013–2018): 4257089.5415293295


### Clean CSV data

In [41]:
df_csv_data["value"] = pd.to_numeric(df_csv_data["value"], errors="coerce")
df_csv_data = df_csv_data[df_csv_data["period"].str.startswith("Q")]


 ### Best Year per Series ID (Max Value Sum)

In [42]:
# Group and sum
grouped = df_csv_data.groupby(["series_id", "year"])["value"].sum().reset_index()

# Find best year for each series_id
best_years = grouped.loc[grouped.groupby("series_id")["value"].idxmax()].reset_index(drop=True)
print(best_years)


             series_id  year    value
0    PRS30006011        2022   20.500
1    PRS30006012        2022   17.100
2    PRS30006013        1998  705.895
3    PRS30006021        2010   17.700
4    PRS30006022        2010   12.400
..                 ...   ...      ...
277  PRS88003192        2002  282.800
278  PRS88003193        2024  861.350
279  PRS88003201        2022   37.200
280  PRS88003202        2022   28.700
281  PRS88003203        2024  583.366

[282 rows x 3 columns]


### Join Time-Series with Population

In [43]:
# Filter for specific series_id and period
filtered_df = df_csv_data[
    (df_csv_data["series_id"].str.strip() == "PRS30006032") &
    (df_csv_data["period"].str.strip() == "Q01")
].copy()

filtered_df["year"] = filtered_df["year"].astype(int)
filtered_df["value"] = pd.to_numeric(filtered_df["value"], errors="coerce")

# Join with population
df_population.rename(columns={"Year": "year", "Population": "population"}, inplace=True)
joined_df = pd.merge(filtered_df, df_population[["year", "population"]], on="year", how="left")

final_report = joined_df[["series_id", "year", "period", "value", "population"]]
print(final_report)

            series_id  year period  value   population
0   PRS30006032        1995    Q01    0.0          NaN
1   PRS30006032        1996    Q01   -4.2          NaN
2   PRS30006032        1997    Q01    2.8          NaN
3   PRS30006032        1998    Q01    0.9          NaN
4   PRS30006032        1999    Q01   -4.1          NaN
5   PRS30006032        2000    Q01    0.5          NaN
6   PRS30006032        2001    Q01   -6.3          NaN
7   PRS30006032        2002    Q01   -6.6          NaN
8   PRS30006032        2003    Q01   -5.7          NaN
9   PRS30006032        2004    Q01    2.0          NaN
10  PRS30006032        2005    Q01   -0.5          NaN
11  PRS30006032        2006    Q01    1.8          NaN
12  PRS30006032        2007    Q01   -0.8          NaN
13  PRS30006032        2008    Q01   -3.5          NaN
14  PRS30006032        2009    Q01  -21.0          NaN
15  PRS30006032        2010    Q01    3.2          NaN
16  PRS30006032        2011    Q01    1.5          NaN
17  PRS300